In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

train_data = pd.read_csv('user-product.csv')
user_item_matrix = train_data.pivot_table(index = 'userID',columns='proID', values = 'ratings', aggfunc='mean').fillna(0)
user_item_matrix

proID,app225748,hkt214024,ind257109,mcl069233,msh218987,spf142809,tbp091787,tvc245565,tvc264525,wcl256284
userID,,,,,,,,,,
0,0.0,0.0,4.6,0.0,0.0,0.0,0.0,4.2,0.0,0.0
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.3,0.0
2,0.0,0.0,0.0,3.7,0.0,5.0,0.0,0.0,0.0,0.0
3,0.0,3.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,3.9
5,0.0,0.0,0.0,0.0,2.8,0.0,0.0,0.0,0.0,3.7
6,0.0,1.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0
7,0.0,0.0,0.0,3.5,0.0,3.3,0.0,0.0,0.0,0.0
8,4.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.9,0.0


In [3]:
user_similarity = cosine_similarity(user_item_matrix)
user_similarity

array([[1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.99575595],
       [0.        , 1.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.9861222 , 0.        ],
       [0.        , 0.        , 1.        , 0.        , 0.        ,
        0.        , 0.        , 0.98424736, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 1.        , 0.        ,
        0.        , 0.96561576, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 1.        ,
        0.99996892, 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.99996892,
        1.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.96561576, 0.        ,
        0.        , 1.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.9842473

In [4]:
target_userID = 9
target_userID_index = user_item_matrix.index.get_loc(target_userID)
target_userID_index

9

In [5]:
user_similarities = user_similarity[target_userID_index]
similar_user_indices = user_similarities.argsort()[::-1][1:]
similar_user_indices

array([0, 8, 7, 6, 5, 4, 3, 2, 1], dtype=int64)

In [9]:
recommend_items = []

for user_index in similar_user_indices:
    rated_by_similar_user = user_item_matrix.iloc[user_index]
    not_rated_by_target_user = (rated_by_similar_user==0) & (user_item_matrix.iloc[target_userID_index]==0)
    
    recommend_items.extend(user_item_matrix.columns[not_rated_by_target_user][:10])

recommend_items_details = train_data[train_data['proID'].isin(recommend_items)][['name','ratings']]
recommend_items_details

,name,ratings
1,USHA Mist AIR DUOS Pedestal Fan (Blue) pack of 4,5.0
2,Urbano Fashion Men's Slim Fit Washed Jeans Stretchable,3.7
3,"Candle Stone Modern Premium Italian Lamps for Home Decoration, Living Room, Bedroom Multiutility Floor Lamp Colour (Blue D...",3.0
4,Aaravin's Comfortable Casual Loafers for Men,3.0
5,RAMDEV Women's Cotton Regular Semi-stitched Kurta (Pack of 3) (RMD_30B_21G_22M_Multicolored_Free Size),3.7
6,"Mallexo Cute Tom and Jerry Keychain for Girls and Boys Set of 2PCs Mickey Mouse Keychain Set for School Bags, Bike, Car et...",2.5
7,"MuscleXP Skipping Rope (Jumping Rope) for Men, Women & Children, Tangle Free Jumping Rope for Kids (Blue/Black)",3.3
8,Techlife Camera Umbrella Holder Clip Clamp Bracket Support for All Professional Tripod,2.9
11,USHA Mist AIR DUOS Pedestal Fan (Blue) pack of 4,4.8
12,Urbano Fashion Men's Slim Fit Washed Jeans Stretchable,3.5


In [18]:
def collaborative_filtering_recommendations(train_data, target_user_id, top_n=10):
    # Create the user-item matrix
    user_item_matrix = train_data.pivot_table(index='userID', columns='proID', values='ratings', aggfunc='mean').fillna(0)

    # Calculate the user similarity matrix using cosine similarity
    user_similarity = cosine_similarity(user_item_matrix)

    # Find the index of the target user in the matrix
    target_user_index = user_item_matrix.index.get_loc(target_user_id)

    # Get the similarity scores for the target user
    user_similarities = user_similarity[target_user_index]

    # Sort the users by similarity in descending order (excluding the target user)
    similar_users_indices = user_similarities.argsort()[::-1][1:]

    # Generate recommendations based on similar users
    recommended_items = []

    for user_index in similar_users_indices:
        # Get items rated by the similar user but not by the target user
        rated_by_similar_user = user_item_matrix.iloc[user_index]
        not_rated_by_target_user = (rated_by_similar_user == 0) & (user_item_matrix.iloc[target_user_index] == 0)

        # Extract the item IDs of recommended items
        recommended_items.extend(user_item_matrix.columns[not_rated_by_target_user][:top_n])

    # Get the details of recommended items
    recommended_items_details = train_data[train_data['proID'].isin(recommended_items)][['name', 'ratings']]

    return recommended_items_details

# Example usage
target_user_id = 8
collaborative_filtering_rec = collaborative_filtering_recommendations(train_data, target_user_id)
print(f"Recommendations for User {target_user_id}:")
collaborative_filtering_rec

Recommendations for User 8:


,name,ratings
0,Bolle & Raven V100 Wired In Ear Earphone with Mic (Black),4.2
2,Urbano Fashion Men's Slim Fit Washed Jeans Stretchable,3.7
3,"Candle Stone Modern Premium Italian Lamps for Home Decoration, Living Room, Bedroom Multiutility Floor Lamp Colour (Blue D...",3.0
4,Aaravin's Comfortable Casual Loafers for Men,3.0
5,RAMDEV Women's Cotton Regular Semi-stitched Kurta (Pack of 3) (RMD_30B_21G_22M_Multicolored_Free Size),3.7
6,"Mallexo Cute Tom and Jerry Keychain for Girls and Boys Set of 2PCs Mickey Mouse Keychain Set for School Bags, Bike, Car et...",2.5
7,"MuscleXP Skipping Rope (Jumping Rope) for Men, Women & Children, Tangle Free Jumping Rope for Kids (Blue/Black)",3.3
9,JAGAN HARDWARE BKR Water Pipe 2-Way Nipple Joint Hose Plastic Black Connector Fitting - Pack of 4 Pieces,3.7
10,Bolle & Raven V100 Wired In Ear Earphone with Mic (Black),2.8
12,Urbano Fashion Men's Slim Fit Washed Jeans Stretchable,3.5
